### CON PLACAS - DIA A DIA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Cargar y preparar los datos
df = pd.read_csv("energy_results.csv")
df['día'] = df['día'].str.extract(r'\d+\s-\s(.+)')[0]
df['día'] = pd.to_datetime(df['día'])
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.drop(columns=['penalizaciones'])
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'solar': 'produccion solar',
    'día': 'fecha'
})

# Separar parte fija y parte animada
fecha_corte = pd.to_datetime("2025-01-01")
df_fijo = df[df['fecha'] <= fecha_corte]
df_animado = df[df['fecha'] > fecha_corte].reset_index(drop=True)

# Crear carpeta para frames del gráfico
os.makedirs("frames_1", exist_ok=True)

# Frame 0 con parte fija
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(df_fijo['fecha'], df_fijo['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
ax1.plot(df_fijo['fecha'], df_fijo['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
ax1.set_xlabel('Año')
ax1.set_ylabel('Euros (€)')
ax1.set_ylim(-5, 15)

ax2 = ax1.twinx()
ax2.plot(df_fijo['fecha'], df_fijo['produccion solar'], label='Producción solar (kW)', color='tab:green')
ax2.set_ylabel('Kilovatios (kW)')
ax2.set_ylim(0, 25)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title('Gastos Energéticos y Producción Solar')
plt.tight_layout()
plt.savefig("frames_1/frame_000.png")
plt.close()

# Generar frames animados
for i in range(1, len(df_animado) + 1):
    df_temp = pd.concat([df_fijo, df_animado.iloc[:i]])

    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax1.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax1.set_xlabel('Año')
    ax1.set_ylabel('Euros (€)')
    ax1.set_ylim(-5, 15)

    ax2 = ax1.twinx()
    ax2.plot(df_temp['fecha'], df_temp['produccion solar'], label='Producción solar (kW)', color='tab:green')
    ax2.set_ylabel('Kilovatios (kW)')
    ax2.set_ylim(0, 25)

    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.title('Gastos Energéticos y Producción Solar')
    plt.tight_layout()
    frame_path = f"frames_1/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_anuales/grafico_energia_con_placas.gif", mode='I', duration=1000) as writer:
    for i in range(len(df_animado) + 1):
        filename = f"frames_1/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil


# Diccionario de nombres de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Cargar y preparar datos
df = pd.read_csv("energy_results.csv")
df['día'] = df['día'].str.extract(r'\d+\s-\s(.+)')[0]
df['día'] = pd.to_datetime(df['día'])
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.drop(columns=['penalizaciones'])
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'solar': 'produccion solar',
    'día': 'fecha'
})

# Calcular ahorro diario
df['ahorro_diario'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Filtrar desde el 1 de enero de 2025
fecha_inicio = pd.to_datetime("2025-01-01")
df = df[df['fecha'] >= fecha_inicio].reset_index(drop=True)

# Crear carpeta para los frames de texto
os.makedirs("frames_2", exist_ok=True)

# Generar imágenes de texto cada 7 días
for i in range(7, len(df), 7):  # cada semana
    datos_7d = df.iloc[i-7:i]
    fecha_dt = df.loc[i-1, 'fecha']
    fecha_actual = f"{fecha_dt.day} de {meses_es[fecha_dt.month]} de {fecha_dt.year}"
    ahorro_7dias = datos_7d['ahorro_diario'].sum()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    texto = f"Dinero ahorrado en los últimos 7 días\nhasta el {fecha_actual}:\n{ahorro_7dias:.2f} €"
    plt.text(0.5, 0.5, texto, ha='center', va='center', fontsize=20, wrap=True,
             transform=plt.gca().transAxes, bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"frames_2/frame_{i:03d}.png")
    plt.close()
# Crear GIF solo de texto (con duración de 70s por frame)
with imageio.get_writer("graficos_anuales/ahorro_texto_con_placas.gif", mode='I', duration=7000) as writer:
    for i in range(7, len(df), 7):
        filename = f"frames_2/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

# (Opcional) eliminar carpeta de imágenes
# shutil.rmtree("frames_2")


C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\912379939.py:59: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


### SIN PLACAS DIA A DIA

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Diccionario de nombres de meses en español
meses_es = {
    1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
    5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
    9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
}

# Cargar y preparar datos
df = pd.read_csv("energy_results_sin_placas.csv")
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'día': 'fecha'
})

# Calcular ahorro diario
df['ahorro_diario'] = df['SIN smartgrids AI'] - df['CON smartgrids AI']

# Filtrar desde el 1 de enero de 2025
fecha_inicio = pd.to_datetime("2025-01-01")
df = df[df['fecha'] >= fecha_inicio].reset_index(drop=True)

# Crear carpeta para los frames de texto sin placas
os.makedirs("frames_2_sin_placas", exist_ok=True)

# Generar imágenes de texto cada 7 días
for i in range(7, len(df), 7):
    datos_7d = df.iloc[i-7:i]
    fecha_dt = df.loc[i-1, 'fecha']
    fecha_actual = f"{fecha_dt.day} de {meses_es[fecha_dt.month]} de {fecha_dt.year}"
    ahorro_7dias = datos_7d['ahorro_diario'].sum()

    fig, ax = plt.subplots(figsize=(8, 4))
    ax.axis('off')

    texto = f"Dinero ahorrado en los últimos 7 días\nhasta el {fecha_actual}:\n{ahorro_7dias:.2f} €"
    plt.text(0.5, 0.5, texto, ha='center', va='center', fontsize=20, wrap=True,
             transform=plt.gca().transAxes, bbox=dict(facecolor='white', edgecolor='black'))

    plt.tight_layout()
    plt.savefig(f"frames_2_sin_placas/frame_{i:03d}.png")
    plt.close()

# Crear GIF solo de texto (con duración de 70s por frame)
with imageio.get_writer("graficos_anuales/ahorro_texto_sin_placas.gif", mode='I', duration=7000) as writer:
    for i in range(7, len(df), 7):
        filename = f"frames_2_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

# (Opcional) eliminar carpeta de imágenes
# shutil.rmtree("frames_2_sin_placas")

C:\Users\anaig\AppData\Local\Temp\ipykernel_22724\1757403973.py:55: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os
import shutil

# Cargar y preparar los datos SIN placas
df = pd.read_csv("energy_results_sin_placas.csv")
df['día'] = pd.date_range(end='2025-05-07', periods=len(df), freq='D')
df = df.rename(columns={
    'baseline': 'SIN smartgrids AI',
    'modelo': 'CON smartgrids AI',
    'día': 'fecha'
})

# Separar parte fija y parte animada
fecha_corte = pd.to_datetime("2025-01-01")
df_fijo = df[df['fecha'] <= fecha_corte]
df_animado = df[df['fecha'] > fecha_corte].reset_index(drop=True)

# Crear carpeta para frames del gráfico sin placas
os.makedirs("frames_1_sin_placas", exist_ok=True)

# Frame 0 con parte fija
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(df_fijo['fecha'], df_fijo['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
ax.plot(df_fijo['fecha'], df_fijo['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
ax.set_xlabel('Año')
ax.set_ylabel('Euros (€)')
ax.set_ylim(-5, 15)
ax.legend(loc='upper left')
plt.title('Gastos Energéticos SIN Placas')
plt.tight_layout()
plt.savefig("frames_1_sin_placas/frame_000.png")
plt.close()

# Generar frames animados
for i in range(1, len(df_animado) + 1):
    df_temp = pd.concat([df_fijo, df_animado.iloc[:i]])

    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df_temp['fecha'], df_temp['SIN smartgrids AI'], label='SIN smartgrids AI', color='tab:blue')
    ax.plot(df_temp['fecha'], df_temp['CON smartgrids AI'], label='CON smartgrids AI', color='tab:orange')
    ax.set_xlabel('Año')
    ax.set_ylabel('Euros (€)')
    ax.set_ylim(-5, 15)
    ax.legend(loc='upper left')
    plt.title('Gastos Energéticos SIN Placas')
    plt.tight_layout()
    frame_path = f"frames_1_sin_placas/frame_{i:03d}.png"
    plt.savefig(frame_path)
    plt.close()

# Crear GIF
with imageio.get_writer("graficos_anuales/grafico_energia_sin_placas.gif", mode='I', duration=1000) as writer:
    for i in range(len(df_animado) + 1):
        filename = f"frames_1_sin_placas/frame_{i:03d}.png"
        image = imageio.imread(filename)
        writer.append_data(image)
